In [4]:
from skyfield.api import EarthSatellite
from skyfield.api import load, wgs84
import numpy as np
import requests

from astropy.time import Time
from astropy import units as u

import spiceypy as sp

In [5]:
# seconds in a day
secperday = 86400
rearthkm = 6370
mu = 398600 #Earth gravitational parameter in km^3/s^2

#This is the skyfield implementation
ts = load.timescale()
eph = load('de430t.bsp')

In [6]:
# Name of the object as apparent in TLE files (CelesTrak)
name = 'ISS'
# Observer latitude (N) in degrees
latitude = 40.1164
# Observer longitude (E) in degrees
longitude = -88.2434
# Observer elevation (m)
elevation = 233
# Start Time, Stop Time and time step of ephemeris output in Julian Date (UT1)

tme = Time('2023-05-21T14:53:0.0',scale='utc')

dt=24/24
#dt=1/86400
jdstart = tme.jd-dt
jdstop = jdstart+dt
jdstep = 1/86400*60*2
#jdstep=1/86400

In [7]:
def get_ephemeris_by_name(name, latitude, longitude, elevation, julian_date):
    '''
    Returns the Right Ascension and Declination relative to the observer's coordinates
    for the given satellite's Two Line Element Data Set at inputted Julian Date.

    **Please note, for the most accurate results, an inputted Julian Date close to the TLE epoch is necessary.

    Parameters
    ---------
    name: 'str'
        CelesTrak name of object
    latitude: 'float'
        The observers latitude coordinate (positive value represents north, negative value represents south)
    longitude: 'float'
        The observers longitude coordinate (positive value represents east, negative value represents west)
    elevation: 'float'
        Elevation in meters
    julian_date: 'float'
        UT1 Universal Time Julian Date. An input of 0 will use the TLE epoch.
    tleapi: 'str'
        base API for query

    Returns
    -------
    Name: 'str'
        The name of the query object
    JulianDate: 'float' or list of 'float'
        UT1 Universal Time Julian Date. 
    Right Ascension: 'float'
        The right ascension of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Declination: 'float'
        The declination of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [-90,90]
    Altitude: 'float'
        The altitude of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,90]
    Azimuth: 'float'
        The azimuth of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Range: 'float'
        Range to object in km
    '''

#    tleLine1, tleLine2 = getTLE(name)
    #ISS (ZARYA)             
    tleLine1='1 25544U 98067A   23107.71565398  .00019656  00000+0  35236-3 0  9995' 
    tleLine2='2 25544  51.6394 268.2146 0006096 203.2981 157.3069 15.49919092392414'
    # #tle = '1 25544U 98067A   23050.90915690  .00019097  00000+0  33852-3 0  9998 2 25544  51.6393 189.4175 0009582   7.6791  73.9158 15.50232555383604'
 


    #Cast the latitude, longitude, and jd to floats (request parses as a string)
    lat = float(latitude)
    lon = float(longitude)
    ele = float(elevation)
    
    # Converting string to list
    jul = str(julian_date).replace("%20", ' ').strip('][').split(', ')
   
    # Converting list elements to float
    jd = [float(i) for i in jul]
   
    #    # return {'jd':jd , "TLELine1":tleLine1, "TLELine2":tleLine2 } 
#    # propagation and create output
#    resultList = []
#    for d in jd:
#        [ra,dec,alt,az,r] = propagateSatellite(tleLine1,tleLine2,lat,lon,ele,d)
#        resultList.append(jsonOutput(name,d,ra,dec,alt,az,r)) 
#    return resultList

    if(len(jd)>1000):
        raise InvalidAPIUsage("Too many entries requested!",status_code=402)   

    # return {'jd':jd , "TLELine1":tleLine1, "TLELine2":tleLine2 } 
    # propagation of satellite position and creation of output
    resultList = []
    for d in jd:
        #Right ascension RA (deg), Declination Dec (deg), dRA/dt*cos(Dec) (deg/day), dDec/dt (deg/day),
        # Altitude (deg), Azimuth (deg), dAlt/dt (deg/day), dAz/dt (deg/day), distance (km), range rate (km/s), phaseangle(deg), illuminated (T/F)   
        [ra, dec, dracosdec, ddec, alt, az, 
         #dalt, daz, 
         r, dr, phaseangle, illuminated] = propagateSatellite(tleLine1,tleLine2,lat,lon,ele,d)
        
        resultList.append(jsonOutput(name, d, ra, dec, dracosdec, ddec,
                                     alt, az, 
                                     #dalt*secperday, daz*secperday, 
                                     r, dr, phaseangle, illuminated)) 
    return resultList

In [8]:
def getTLE(targetName, tleapi='https://celestrak.org/NORAD/elements/gp.php?NAME='):
    """
    Query Two Line Element (orbital element) API and return TLE lines for propagation
    
    Paremeters:
    ------------
    targetName: 'str'
        Name of satellite as displayed in TLE file
    tleapi: 'str'
        URL of TLE API
        
        
    Returns:
    --------
    tleLine1: 'str'
        TLE line 1
    tleLine2: 'str'
        TLE line 2
    """

    # uncomment if json output is required
    #tleapiResult=requests.get(f'{tleapi}{targetName}&FORMAT=JSON').json()	    

    # we will go with the standard TLE format here
    tleapiResult=requests.get(f'{tleapi}{targetName}&FORMAT=TLE')
    
    tle = tleapiResult.text.replace("%20", ' ')
    #Retrieve the two lines
    tleLine1 = tle[26:95]
    tleLine2 = tle[97:166]

    return tleLine1, tleLine2


def propagateSatellite(tleLine1, tleLine2, lat, lon, elevation, jd, dtsec=1):
    """Use Skyfield (https://rhodesmill.org/skyfield/earth-satellites.html) 
     to propagate satellite and observer states.
     
     Parameters
    ---------
    tleLine1: 'str'
        TLE line 1
    tleLine2: 'str'
         TLE line 2
    lat: 'float'
        The observer WGS84 latitude in degrees
    lon: 'float'
        The observers WGS84 longitude in degrees (positive value represents east, negatie value represents west)
    elevation: 'float'
        The observer elevation above WGS84 ellipsoid in meters
    julian_date: 'float'
        UT1 Universal Time Julian Date. An input of 0 will use the TLE epoch.
    tleapi: 'str'
        base API for query

    Returns
    -------
    Right Ascension: 'float'
        The right ascension of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    Declination: 'float'
        The declination of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [-90,90]
    Altitude: 'float'
        The altitude of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,90]
    Azimuth: 'float'
        The azimuth of the satellite relative to observer coordinates in ICRS reference frame in degrees. Range of response is [0,360)
    distance: 'float'
        Range from observer to object in km
    """
        

    ts = load.timescale()
    satellite = EarthSatellite(tleLine1,tleLine2,ts = ts)

    #Get current position and find topocentric ra and dec
    currPos = wgs84.latlon(lat, lon, elevation)
    # Set time to satellite epoch if input jd is 0, otherwise time is inputted jd
    if jd == 0: t = ts.ut1_jd(satellite.model.jdsatepoch)
    else: t = ts.ut1_jd(jd)

    difference = satellite - currPos
    topocentric = difference.at(t)
    topocentricn = topocentric.position.km/np.linalg.norm(topocentric.position.km)
    
    ra, dec, distance = topocentric.radec()
    alt, az, distance = topocentric.altaz()
    
    dtday=dtsec/secperday
    tplusdt = ts.ut1_jd(jd+dtday)
    tminusdt = ts.ut1_jd(jd-dtday)
    
#     vtopo = difference.at(dtplus)-difference.at(dtminus)
#     print(topocentric)
#     print(vtopo)
#     dradt, ddecdt, ddistancedt = vtopo.radec()
    
    dtx2 = 2*dtsec 
    print(dtx2)
#     dra = dradt.value/dtx2
#     ddec = ddecdt.value/dtx2
#     ddistance = ddistancedt.km/dtx2 
    print(satellite.at(t).position.km)
    sat = satellite.at(t).position.km
    print('sat',sat)
    satn = sat/np.linalg.norm(sat)
    satpdt = satellite.at(tplusdt).position.km
    satmdt = satellite.at(tminusdt).position.km
    vsat = (satpdt - satmdt)/dtx2
    print('vsat',vsat)
    
    sattop = difference.at(t).position.km
    sattopr = np.linalg.norm(sattop)
    sattopn = sattop/sattopr
    sattoppdt = difference.at(tplusdt).position.km
    sattopmdt = difference.at(tminusdt).position.km
    
    ratoppdt,dectoppdt = icrf2radec(sattoppdt)
    ratopmdt,dectopmdt = icrf2radec(sattopmdt)
    
    vsattop = (sattoppdt - sattopmdt)/dtx2
    
    ddistance = np.dot(vsattop,sattopn)
    print('sattop',sattop)
    print('vsattop',vsattop)
    print('vsattop/sattopr',vsattop/sattopr)
    rxy = np.dot(sattop[0:2],sattop[0:2])
    dra = (sattop[1]*vsattop[0]-sattop[0]*vsattop[1])/rxy
    print('x^2+y^2', rxy)
    ddec = vsattop[2]/np.sqrt(1-sattopn[2]*sattopn[2])
    dracosdec = dra*np.cos(dec.radians)
    print('from xyz: dra,ddec,ddistance,dracosdec',[dra,ddec,ddistance,dracosdec])
 
    dra = (ratoppdt - ratopmdt)/dtx2
    ddec = (dectoppdt - dectopmdt)/dtx2
    dracosdec = dra*np.cos(dec.radians)
    print('from r: dra,ddec,ddistance,dracosdec',[dra,ddec,ddistance,dracosdec])
    
    drav, ddecv = uicrf2radec(vsattop/sattopr)
    dracosdecv = drav*np.cos(dec.radians)
    
    print('from v: dra,ddec,ddistance,dracosdec',[drav,ddecv,ddistance,dracosdecv])
    
    
    earth = eph['Earth']
    sun = eph['Sun']
    print(t,ts.ut1(jd))
    earthp = earth.at(ts.ut1_jd(jd)).position.km
    sunp = sun.at(ts.ut1_jd(jd)).position.km
    earthsun = sunp - earthp
    earthsunn = earthsun/np.linalg.norm(earthsun)
    satsun =  sat - earthsun
    satsunn = satsun/np.linalg.norm(satsun)
    phase_angle = np.rad2deg(np.arccos(np.dot(satsunn,topocentricn)))
    
    #Is the satellite in Earth's Shadow?
    r_parallel = np.dot(sat,earthsunn)*earthsunn
    r_tangential = sat-r_parallel

    if alt.degrees>0:
        print('Satellite is above the horizon')
    else:
        print('Satellite is not above the horizon')

    illuminated = True

    if(np.linalg.norm(r_parallel)<0):
        if(np.linalg.norm(r_tangential)<rearthkm):
            #print(np.linalg.norm(r_tangential),np.linalg.norm(r))
            #yes the satellite is in Earth's shadow, no need to continue (except for the moon of course)
            illuminated = False
    
    return (ra, dec, dracosdec, ddec, alt, az, 
            distance, ddistance, phase_angle, illuminated)


def my_arange(a, b, dr, decimals=11):
    """
    Better arange function that compensates for round-off errors.
    
    Parameters:
    -----------
    a: 'float'
        first element in range 
    b: 'float'
        last element in range
    dr: 'float'
        range increment
    decimals: 'integer'
        post comma digits to be rounded to
        
    Returns:
    --------
    res: 'numpy array of floats'
        array of numbers between a and b with dr increments
    """
    
    res = [a]
    k = 1
    while res[-1] < b:
        tmp = np.round(a + k*dr,decimals)
        if tmp > b:
            break   
        res.append(tmp)
        k+=1

    return np.asarray(res) 

def tle2ICRFstate(tleLine1,tleLine2,jd):

    #This is the skyfield implementation
    ts = load.timescale()
    satellite = EarthSatellite(tleLine1,tleLine2,ts = ts)

    # Set time to satellite epoch if input jd is 0, otherwise time is inputted jd
    if jd == 0: t = ts.ut1_jd(satellite.model.jdsatepoch)
    else: t = ts.ut1_jd(jd)

    r =  satellite.at(t).position.km
    # print(satellite.at(t))
    v = satellite.at(t).velocity.km_per_s
    return np.concatenate(np.array([r,v]))

def jsonOutput(name,time,ra,dec,dracosdec,ddec, 
               alt, az, 
               #dalt, daz, 
               r, dr, phaseangle, illuminated, 
               precisionAngles=11,precisionDate=12,precisionRange=12):
    """
    Convert API output to JSON format
    
    Parameters:
    -----------
    name: 'str'
        Name of the target satellite
    time: 'float'
        Julian Date
    ra: Skyfield object / 'float'
        Right Ascension 
    dec: Skyfield object / 'float'
        Declination
    alt: Skyfield object / 'float'
        Altitude
    az: Skyfield object / 'float'
        Azimuth
    r: Skyfield object / 'float'
        Range to target
    precisionAngles: 'integer'
        number of digits for angles to be rounded to (default: micro arcsec)
    precisionDate: 'integer'
        number of digits for Julian Date to be rounded to (default: micro sec)
    precisionRange: 'integer'
        number of digits for angles to be rounded to (default: nano meters)   
        
    Returns:
    --------
    output: 'dictionary'
        JSON dictionary of the above quantities
    
    """
    
    #looking up the numpy round function once instead of multiple times makes things a little faster
    myRound = np.round
    output= {"NAME": name,
            "JULIAN_DATE": myRound(time,precisionDate),
            "RIGHT_ASCENSION-DEG": myRound(ra._degrees,precisionAngles),
            "DECLINATION-DEG": myRound(dec.degrees,precisionAngles),
            "DRA_COSDEC-DEG_PER_SEC":  myRound(dracosdec,precisionAngles),
            "DDEC-DEG_PER_SEC": myRound(ddec,precisionAngles),
            "ALTITUDE-DEG": myRound(alt.degrees,precisionAngles),
            "AZIMUTH-DEG": myRound(az.degrees,precisionAngles),
            # "DALT-DEG_PER_SEC": myRound(dalt,precisionAngles),
            # "DAZ-DEG_PER_SEC": myRound(daz, precisionAngles),
            "RANGE-KM": myRound(r.km,precisionRange),
            "RANGE_RATE-KM_PER_SEC": myRound(dr,precisionRange),
            "PHASE_ANGLE-DEG": myRound(phaseangle, precisionAngles),
            "ILLUMINATED": illuminated
            } 
    
    return output   


def icrf2radec(pos, deg=True):
    """
    Convert ICRF xyz to Right Ascension and Declination.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    pos ... real, dim=[n, 3], 3D vector of unit length (ICRF)
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    """
    norm=np.linalg.norm
    array=np.array
    arctan2=np.arctan2
    arcsin=np.arcsin
    rad2deg=np.rad2deg
    modulo=np.mod
    pix2=2.*np.pi
    
    if(pos.ndim>1):
        r=norm(pos,axis=1)
        xu=pos[:,0]/r
        yu=pos[:,1]/r
        zu=pos[:,2]/r
    else:
        r=norm(pos)
        xu=pos[0]/r
        yu=pos[1]/r
        zu=pos[2]/r
    
    phi=arctan2(yu,xu)
    delta=arcsin(zu)
    
    if(deg):
        ra = modulo(rad2deg(phi)+360,360)
        dec = rad2deg(delta)
    else:
        ra = modulo(phi+pix2,pix2)
        dec = delta
    
    return ra, dec


def uicrf2radec(pos, deg=True):
    """
    Convert ICRF xyz unit vector to Right Ascension and Declination.
    Geometric states on unit sphere, no light travel time/aberration correction.
    
    Parameters:
    -----------
    pos ... real, dim=[n, 3], 3D vector of unit length (ICRF)
    deg ... True: angles in degrees, False: angles in radians
    Returns:
    --------
    ra ... Right Ascension [deg]
    dec ... Declination [deg]
    """
    norm=np.linalg.norm
    array=np.array
    arctan2=np.arctan2
    arcsin=np.arcsin
    rad2deg=np.rad2deg
    modulo=np.mod
    pix2=2.*np.pi
    
    if(pos.ndim>1):
        xu=pos[:,0]
        yu=pos[:,1]
        zu=pos[:,2]
    else:
        xu=pos[0]
        yu=pos[1]
        zu=pos[2]
    
    phi=arctan2(yu,xu)
    delta=arcsin(zu)
    
    if(deg):
        ra = modulo(rad2deg(phi)+360,360)
        dec = rad2deg(delta)
    else:
        ra = modulo(phi+pix2,pix2)
        dec = delta
    
    return ra, dec

In [9]:
get_ephemeris_by_name(name, latitude, longitude, elevation, 2460000.5)

2
[ 5782.67951672 -3129.49968191  1747.10975246]
sat [ 5782.67951672 -3129.49968191  1747.10975246]
vsat [ 3.67223554  3.59602288 -5.67414416]
sattop [ 3791.65322941 -7594.0488384  -2336.34379313]
vsattop [ 3.99778357  3.45150058 -5.67486603]
vsattop/sattopr [ 0.0004541   0.00039205 -0.0006446 ]
x^2+y^2 72046211.97208774
from xyz: dra,ddec,ddistance,dracosdec [-0.0006030331890485428, -5.885916511511101, 0.25055249767935717, -0.000581410312945243]
from r: dra,ddec,ddistance,dracosdec [0.034551264712348484, -0.03785768337077311, 0.25055249767935717, 0.0333123648812029]
from v: dra,ddec,ddistance,dracosdec [40.80580074222644, -0.036932968547061545, 0.25055249767935717, 39.342632893807945]
<Time tt=2460000.500800916> <Time tt=900445196.137256>
Satellite is not above the horizon


[{'NAME': 'ISS',
  'JULIAN_DATE': 2460000.5,
  'RIGHT_ASCENSION-DEG': 296.53262225061,
  'DECLINATION-DEG': -15.38969708717,
  'DRA_COSDEC-DEG_PER_SEC': 0.03331236488,
  'DDEC-DEG_PER_SEC': -0.03785768337,
  'ALTITUDE-DEG': -39.75361675003,
  'AZIMUTH-DEG': 284.539712562,
  'RANGE-KM': 8803.676180537454,
  'RANGE_RATE-KM_PER_SEC': 0.250552497679,
  'PHASE_ANGLE-DEG': 139.5195187027,
  'ILLUMINATED': True}]

In [10]:
get_ephemeris_by_name(name, latitude, longitude, elevation, 2460000.5+1/86400)

2
[ 5786.34808889 -3125.90167649  1741.43449832]
sat [ 5786.34808889 -3125.90167649  1741.43449832]
vsat [ 3.66490649  3.59998568 -5.67636051]
sattop [ 3795.64735488 -7590.59534341 -2342.01976915]
vsattop [ 3.99046506  3.45548712 -5.67708241]
vsattop/sattopr [ 0.00045326  0.00039249 -0.00064484]
x^2+y^2 72024076.51005974
from xyz: dra,ddec,ddistance,dracosdec [-0.0006026570296180056, -5.889287949332287, 0.25136884156167927, -0.0005809418135438553]
from r: dra,ddec,ddistance,dracosdec [0.034529712363081444, -0.0378759198327856, 0.25136884156167927, 0.03328552183996111]
from v: dra,ddec,ddistance,dracosdec [40.890465475565804, -0.03694633989247756, 0.25136884156167927, 39.417081362320964]
<Time tt=2460000.50081249> <Time tt=900445196.137256>
Satellite is not above the horizon


[{'NAME': 'ISS',
  'JULIAN_DATE': 2460000.5000115735,
  'RIGHT_ASCENSION-DEG': 296.56716273747,
  'DECLINATION-DEG': -15.42756389672,
  'DRA_COSDEC-DEG_PER_SEC': 0.03328552184,
  'DDEC-DEG_PER_SEC': -0.03787591983,
  'ALTITUDE-DEG': -39.7553028758,
  'AZIMUTH-DEG': 284.47750568821,
  'RANGE-KM': 8803.927141289976,
  'RANGE_RATE-KM_PER_SEC': 0.251368841562,
  'PHASE_ANGLE-DEG': 139.55038017398,
  'ILLUMINATED': True}]